In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append('../src/data')
sys.path.append('../src/features')
sys.path.append('../src/models')
sys.path.append('../src/visualization')
from utils import *
from data_module import *
from model import *
from dataset import *
from visualization import *

In [3]:
plt.rcParams['figure.figsize'] = (14,10)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 22}
plt.rc('font', **font)

In [4]:
df = pd.read_csv('~/work/interim/dataset_strat.csv')

In [5]:
df["bboxes"] = df["bboxes"].apply(eval) # DONT forget this cell

In [6]:
#dm = BoltDataModule(df, num_workers = 16, bs = 2)

In [7]:
model = BoltDetector(df, bs=3, num_classes=1, img_size=1024)

{'name': 'tf_efficientnetv2_l', 'backbone_name': 'tf_efficientnetv2_l', 'backbone_args': {'drop_path_rate': 0.2}, 'backbone_indices': None, 'image_size': [1024, 1024], 'num_classes': 1, 'min_level': 3, 'max_level': 7, 'num_levels': 5, 'num_scales': 3, 'aspect_ratios': [[1.0, 1.0], [1.4, 0.7], [0.7, 1.4]], 'anchor_scale': 4.0, 'pad_type': 'same', 'act_type': 'swish', 'norm_layer': None, 'norm_kwargs': {'eps': 0.001, 'momentum': 0.01}, 'box_class_repeats': 3, 'fpn_cell_repeats': 3, 'fpn_channels': 88, 'separable_conv': True, 'apply_resample_bn': True, 'conv_bn_relu_pattern': False, 'downsample_type': 'max', 'upsample_type': 'nearest', 'redundant_bias': True, 'head_bn_level_first': False, 'head_act_type': None, 'fpn_name': None, 'fpn_config': None, 'fpn_drop_path_rate': 0.0, 'alpha': 0.25, 'gamma': 1.5, 'label_smoothing': 0.0, 'legacy_focal': False, 'jit_loss': False, 'delta': 0.1, 'box_loss_weight': 50.0, 'soft_nms': False, 'max_detection_points': 5000, 'max_det_per_image': 100, 'url': '

In [8]:
#path = '/media/scratch/astamoulakatos/NMIS/detector/detector-epoch=83-valid_loss=0.57.ckpt'
# Get the latest model
#path = '/media/scratch/astamoulakatos/NMIS/detector/detector-epoch=83-valid_loss=0.57.ckpt'
import os
detector="../../bolt_logs/detector"
for root, dirs, files in os.walk(detector, topdown=False):
  print(files)
  for name in files:
    path= os.path.join(detector, name)
    break

print(path)

[]
['epoch=14-valid_loss=0.44.ckpt', 'epoch=13-valid_loss=0.48.ckpt']
../../bolt_logs/detector/epoch=14-valid_loss=0.44.ckpt


In [9]:
model = model.load_from_checkpoint(path, map_location='cuda:0')

{'name': 'tf_efficientnetv2_l', 'backbone_name': 'tf_efficientnetv2_l', 'backbone_args': {'drop_path_rate': 0.2}, 'backbone_indices': None, 'image_size': [1024, 1024], 'num_classes': 1, 'min_level': 3, 'max_level': 7, 'num_levels': 5, 'num_scales': 3, 'aspect_ratios': [[1.0, 1.0], [1.4, 0.7], [0.7, 1.4]], 'anchor_scale': 4.0, 'pad_type': 'same', 'act_type': 'swish', 'norm_layer': None, 'norm_kwargs': {'eps': 0.001, 'momentum': 0.01}, 'box_class_repeats': 3, 'fpn_cell_repeats': 3, 'fpn_channels': 88, 'separable_conv': True, 'apply_resample_bn': True, 'conv_bn_relu_pattern': False, 'downsample_type': 'max', 'upsample_type': 'nearest', 'redundant_bias': True, 'head_bn_level_first': False, 'head_act_type': None, 'fpn_name': None, 'fpn_config': None, 'fpn_drop_path_rate': 0.0, 'alpha': 0.25, 'gamma': 1.5, 'label_smoothing': 0.0, 'legacy_focal': False, 'jit_loss': False, 'delta': 0.1, 'box_loss_weight': 50.0, 'soft_nms': False, 'max_detection_points': 5000, 'max_det_per_image': 100, 'url': '

In [10]:
model.eval();

In [11]:
loader = model.val_dataloader()

In [12]:
outputs = []
for batch in loader:
    output = model.validation_step(batch=batch, batch_idx=0)
    outputs.append(output)

/opt/conda/lib/python3.10/site-packages/effdet-0.3.0-py3.10.egg/effdet/bench.py:55: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  indices_all = cls_topk_indices_all // num_classes
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/core/module.py:377: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(
/opt/conda/lib/python3.10/site-packages/effdet-0.3.0-py3.10.egg/effdet/bench.py:55: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds

In [13]:
len(outputs)

18

In [14]:
model.validation_epoch_end(outputs)

creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.30s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.405
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.564
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.522
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.405
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.004
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.043
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.435
 Average Recall     (AR) @[ IoU=0

{'val_loss': tensor(0.4437),
 'metrics': {'AP_all': 0.40538368913941003,
  'AP_all_IOU_0_50': 0.5637623762376237,
  'AP_all_IOU_0_75': 0.5224696843834202,
  'AP_small': -1.0,
  'AP_medium': 0.40538368913941003,
  'AP_large': -1.0,
  'AR_all_dets_1': 0.004,
  'AR_all_dets_10': 0.04342857142857143,
  'AR_all': 0.43485714285714294,
  'AR_small': -1.0,
  'AR_medium': 0.43485714285714294,
  'AR_large': -1.0}}

In [15]:
loader = model.test_dataloader()

In [16]:
outputs = []
for batch in loader:
    output = model.validation_step(batch=batch, batch_idx=0)
    outputs.append(output)

In [17]:
model.validation_epoch_end(outputs)

creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.26s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.384
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.552
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.467
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.384
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.005
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.044
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.422
 Average Recall     (AR) @[ IoU=0

{'val_loss': tensor(0.3750),
 'metrics': {'AP_all': 0.38351518893288955,
  'AP_all_IOU_0_50': 0.5516518195937241,
  'AP_all_IOU_0_75': 0.4668547640708516,
  'AP_small': -1.0,
  'AP_medium': 0.38351518893288955,
  'AP_large': -1.0,
  'AR_all_dets_1': 0.005389221556886228,
  'AR_all_dets_10': 0.04431137724550898,
  'AR_all': 0.42215568862275443,
  'AR_small': -1.0,
  'AR_medium': 0.42215568862275443,
  'AR_large': -1.0}}

In [18]:
model.device

device(type='cpu')

In [19]:
output['batch_predictions']

{'predictions': tensor([[[8.8570e+02, 1.5380e+02, 9.3887e+02, 2.2195e+02, 7.3911e-01,
           1.0000e+00],
          [8.3846e+01, 3.6645e+02, 1.3663e+02, 4.3459e+02, 7.3621e-01,
           1.0000e+00],
          [8.8562e+02, 6.0749e+02, 9.3834e+02, 6.7525e+02, 7.2081e-01,
           1.0000e+00],
          ...,
          [8.5859e+02, 8.3636e+02, 1.0240e+03, 9.4945e+02, 4.5974e-02,
           1.0000e+00],
          [8.2359e+01, 3.7756e+02, 1.0848e+02, 4.0890e+02, 4.5927e-02,
           1.0000e+00],
          [9.0192e+02, 1.7062e+02, 9.3013e+02, 2.1373e+02, 4.5920e-02,
           1.0000e+00]],
 
         [[3.6834e+02, 8.1228e+02, 4.2132e+02, 8.8004e+02, 7.6193e-01,
           1.0000e+00],
          [9.1263e+02, 1.0242e+02, 9.6568e+02, 1.7049e+02, 7.5736e-01,
           1.0000e+00],
          [9.2127e+01, 1.0725e+02, 1.4517e+02, 1.7613e+02, 7.5618e-01,
           1.0000e+00],
          ...,
          [9.4755e+01, 7.7762e+02, 1.4478e+02, 8.5007e+02, 6.7075e-02,
           1.0000e+00],
  